# Memory


In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.language_models import FakeListChatModel
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import trim_messages, HumanMessage


class PrintOutputCallback(BaseCallbackHandler):
    def on_chat_model_start(self, serialized, messages, **kwargs) -> None:
        print(f"Amount of input messages: {len(messages)}")


sessions: dict[str, InMemoryChatMessageHistory] = {}
handler = PrintOutputCallback()
llm = FakeListChatModel(responses=["ai1", "ai2", "ai3"])


def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in sessions:
        sessions[session_id] = InMemoryChatMessageHistory()
    return sessions[session_id]


trimmer = trim_messages(
    max_tokens=1,
    strategy="last",
    token_counter=len,
    include_system=True,
    start_on="human",
)

raw_chain = trimmer | llm
chain = RunnableWithMessageHistory(raw_chain, get_session_history)

config = {"callbacks": [PrintOutputCallback()], "configurable": {"session_id": "1"}}
_ = chain.invoke([HumanMessage("Hi!")], config=config)

print(f"History length: {len(sessions['1'].messages)}")

_ = chain.invoke(
    [HumanMessage("How are you?")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

Amount of input messages: 1
History length: 2
Amount of input messages: 1
History length: 4


In [3]:
sessions["1"].messages

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai1', additional_kwargs={}, response_metadata={}, id='run--d2f77316-0821-404a-a8e2-672890c92e5e-0'),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai2', additional_kwargs={}, response_metadata={}, id='run--e433f818-682d-4e24-8c74-0a849df68400-0')]

In [4]:
trimmer.invoke(sessions["1"].messages)

[]

In [13]:
from langgraph.graph import MessageGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage


def test_node(state):
    print(f"State: {state}")
    print(f"History length= {len(state[:-1])}")
    return [AIMessage("Hello!")]


builder = MessageGraph()
builder.add_node("test_node", test_node)
builder.add_edge(START, "test_node")
builder.add_edge("test_node", END)
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [15]:
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}}
)
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-b"}}
)
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}}
)

State: [HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='a27eb021-2e2e-4598-9847-106e1d0ffdb7'), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='e732eff6-553c-47a4-8b6e-fe37e67b52e7'), HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='ee99f495-a3b3-494e-81ad-3585ff3d79cf'), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='284e0048-d8f9-4de1-b313-eb10895c543d'), HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='7291cc37-1556-41d9-9172-2d01e6dfb6a3')]
History length= 4
State: [HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='e1e52d07-007f-4c63-9c62-b6989380cf44'), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='94a6217d-f1e7-4f7f-afb6-6ad1f12856a2'), HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='66336fb7-3cba-4c2e-9733-c9d434e689c5')]
History length= 2
State: [HumanMe

In [12]:
checkpoints = list(memory.list(config={"configurable": {"thread_id": "thread-a"}}))

for check_point in checkpoints:
    print(check_point.config["configurable"]["checkpoint_id"])

1f04e9d2-bc0e-680c-800d-2594d65b57d6
1f04e9d2-bc0d-66fa-800c-75c49ef1c243
1f04e9d2-bc0c-6692-800b-7f063232758e
1f04e9d2-bc05-6a04-800a-00cb1962950a
1f04e9d2-bc03-6970-8009-2741c872bd40
1f04e9d2-bc00-6ec8-8008-7225e723a464
1f04e9d2-500a-63e0-8007-d7f364ecbd2e
1f04e9d2-5009-6558-8006-5b5944f07b16
1f04e9d2-5008-605e-8005-4b47c684d857
1f04e9d2-5001-67cc-8004-8768c80060a8
1f04e9d2-4fff-62ba-8003-2e8d0a51ac4d
1f04e9d2-4ffc-64a2-8002-5cede3fb100e
1f04e9d1-8e5c-6f72-8001-8e4abfd66ad2
1f04e9d1-8e5a-6c36-8000-e2f7bcdb41a0
1f04e9d1-8e57-6b26-bfff-9f861d1f04bf
